In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [5]:
# Load Data (replace this with actual CSV file or DataFrame)
data = pd.read_csv('merged_properties.csv')  # Replace 'your_file.csv' with your actual file path
# Features and targets
# X = data.drop(columns=['c44', 'e15', 'q15', 'μ11', 'ϵ11', 'α11', 'ρ']).values
X = data[['vf', 'c55e', 'e15e', 'q15e', 'ϵ11e', 'μ11e', 'α11e', 'ρe', 'c55f', 'e15f', 'q15f', 'ϵ11f', 'μ11f', 'α11f', 'ρf']].values
y = data[['c44', 'e15', 'q15', 'μ11', 'ϵ11', 'α11', 'ρ']].values

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

# Split first
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scalers_X = []
X_train_scaled = np.zeros_like(X_train)
X_test_scaled = np.zeros_like(X_test)

for i in range(X.shape[1]):
    scaler = MinMaxScaler()
    X_train_scaled[:, i] = scaler.fit_transform(X_train[:, i].reshape(-1, 1)).flatten()
    X_test_scaled[:, i] = scaler.transform(X_test[:, i].reshape(-1, 1)).flatten()
    scalers_X.append(scaler)

# Scale y column-wise
scalers_y = []
y_train_scaled = np.zeros_like(y_train)
y_test_scaled = np.zeros_like(y_test)

for i in range(y.shape[1]):
    scaler = MinMaxScaler()
    y_train_scaled[:, i] = scaler.fit_transform(y_train[:, i].reshape(-1, 1)).flatten()
    y_test_scaled[:, i] = scaler.transform(y_test[:, i].reshape(-1, 1)).flatten()
    scalers_y.append(scaler)

print(f"Shape of X_train_scaled: {scalers_X.shape}")
print(f"Shape of X_test_scaled: {scalers_y.shape}")

Shape of X: (1045, 15)
Shape of y: (1045, 7)


AttributeError: 'list' object has no attribute 'shape'

In [6]:
def weighted_mse(y_true, y_pred):
    weights = 1.0 / (tf.abs(y_true) + 1e-6)  # Use TensorFlow ops only
    squared_diff = tf.square(y_true - y_pred)
    loss = tf.reduce_mean(weights * squared_diff)
    return loss

model = Sequential([
    Dense(64, activation='relu', input_shape=(15,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(7, activation='linear')  # 7 target outputs
])

# Compile with standard MSE
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Training
model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_test_scaled, y_test_scaled),
    epochs=300,
    batch_size=32,
    verbose=1
)

Epoch 1/300


/opt/anaconda3/envs/Thesis/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2465 - mae: 0.4087 - val_loss: 0.0260 - val_mae: 0.1189
Epoch 2/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0232 - mae: 0.1154 - val_loss: 0.0132 - val_mae: 0.0799
Epoch 3/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0120 - mae: 0.0775 - val_loss: 0.0084 - val_mae: 0.0632
Epoch 4/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0077 - mae: 0.0617 - val_loss: 0.0058 - val_mae: 0.0508
Epoch 5/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0051 - mae: 0.0484 - val_loss: 0.0041 - val_mae: 0.0400
Epoch 6/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0038 - mae: 0.0391 - val_loss: 0.0033 - val_mae: 0.0338
Epoch 7/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0025 - mae: 0.0316 - val_loss: 0.0027 - val_mae: 0.0305
Epoch 8/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0019 - mae: 0.0272 - val_loss: 0.0023 - val_mae: 0.0255
Epoch 9/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0016 - mae: 0.0

In [7]:
# Evaluate model
loss, mae = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
print(f"Test MAE: {mae:.4f}")

Test MAE: 0.0104


In [8]:
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform column-wise
y_pred = np.zeros_like(y_pred_scaled)
for i in range(y.shape[1]):
    y_pred[:, i] = scalers_y[i].inverse_transform(y_pred_scaled[:, i].reshape(-1, 1)).flatten()

np.set_printoptions(linewidth=np.inf)  # Set the print options to avoid line breaks
for i in range(10):
    print(f"Predicted: {y_pred[i]}")
    print(f"Actual:    {y_test[i]}")
    print("-" * 50)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Predicted: [-4.2455405e+08  1.0124809e+01  2.2799004e+01  2.8457589e-05 -9.7032979e-02 -9.5848307e-09  7.2686123e+03]
Actual:    [ 5.75659234e+10  3.79152653e-02  1.01572435e+02  3.00662118e-05  4.87692269e-10 -6.17503512e-13  7.33000000e+03]
--------------------------------------------------
Predicted: [ 9.9034218e+10 -4.2887077e+00  2.6489613e+01  7.4703875e-04  4.7697514e-02  3.3607481e-09  7.0516479e+03]
Actual:    [9.65571615e+10 5.65677431e-01 1.46347462e-03 7.18733615e-04 7.40538530e-10 2.87408973e-10 7.10000000e+03]
--------------------------------------------------
Predicted: [ 9.5527223e+10 -9.6906872e+00  1.5743204e+00  3.1876080e-03  6.6681594e-02  1.0846124e-09  5.1608896e+03]
Actual:    [3.06869106e+10 3.47867300e-04 1.45706883e-06 3.18686247e-03 1.43839504e-10 7.08191652e-10 5.28000000e+03]
--------------------------------------------------
Predicted: [ 3.8512280e+11  1.1251884e+02  1.5448137e+02  3.8083432e-05  6.6531166e-02 -3.0999

In [9]:
# -------- Prediction Function --------
def predict_custom_input(input_list):
    """
    input_list: list of 15 feature values in order
    """
    if len(input_list) != 15:
        raise ValueError("Input must be a list of 15 feature values.")
    
    input_scaled = scaler_X.transform([input_list])
    prediction_scaled = model.predict(input_scaled, verbose=0)
    prediction = scaler_y.inverse_transform(prediction_scaled)
    
    # Display the results
    target_columns = [
        "c55", "e15", "q15",
        "ϵ11", "μ11", "α11", "ρ"
    ]
    result = dict(zip(target_columns, prediction[0]))
    print("Predicted Properties:")
    for k, v in result.items():
        print(f"{k}: {v:.4f}")
    return prediction[0]

# -------- Example of using predict_custom_input --------
# Example Input (replace these with actual values when running)
example_input = [
    0.05, 44000000000.0, 11.4, 0.0, 9.82e-09, 5e-06, 0, 5700, 45300000000.0, 0, 550.0, 8.000000000000001e-11, 0.000157, 0.0, 5300
]

example_output = [
    47665708776.4155, 0.015891104909231, 496.3562235447875, 0.0001422813709311, 9.727373332703483e-11, -4.046667731750246e-12, 5320.0
]

# Get the prediction
predicted_output = predict_custom_input(example_input)

# Compare with example_output
print("\nComparison with Example Output:")
for i, (predicted, example) in enumerate(zip(predicted_output, example_output)):
    print(f"Target {i + 1}: Predicted = {predicted:.4f}, Example = {example:.4f}, Difference = {abs(predicted - example):.4f}")

NameError: name 'scaler_X' is not defined